### LCEL Deepdive

In [86]:
!pip install -qU langchain-groq langchain_community langchain_huggingface faiss-cpu

In [87]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
#from dotenv import load_dotenv
#load_dotenv()
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [88]:
prompt = ChatPromptTemplate.from_template("Explain about {topic} in detail")
#model = ChatOpenAI()
# Initialize the Groq model
model = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2,
)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

'Ice cream is a frozen dessert made from a mixture of cream, sugar, and flavorings, typically frozen to a smooth and creamy consistency. It has been a popular treat for centuries, with its origins dating back to ancient civilizations in the Middle East and China.\n\n**History of Ice Cream**\n\nThe earliest known evidence of ice cream-like desserts dates back to ancient Mesopotamia, around 2000 BC. The ancient Greeks and Romans also enjoyed frozen desserts made from snow and sweetened with honey. However, the modern version of ice cream as we know it today originated in Italy in the 16th century.\n\nThe Medici family in Florence, Italy, commissioned a chef named Bernardo Buontalenti to create a frozen dessert for a banquet. Buontalenti created a frozen mixture of cream, sugar, and fruit, which became known as "gelato." The name "gelato" is still used today to describe Italian-style ice cream.\n\n**Ingredients of Ice Cream**\n\nIce cream is typically made from a combination of the follow

In [89]:
print(prompt.invoke({"topic": "ice cream"}))

messages=[HumanMessage(content='Explain about ice cream in detail', additional_kwargs={}, response_metadata={})]


In [90]:
from langchain_core.messages.human import HumanMessage

messages = [HumanMessage(content='tell me a short joke about ice cream')]
model.invoke(messages)

AIMessage(content='Why did the ice cream go to the doctor? \n\nBecause it had a meltdown.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 43, 'total_tokens': 61, 'completion_time': 0.026576425, 'completion_tokens_details': None, 'prompt_time': 0.00279619, 'prompt_tokens_details': None, 'queue_time': 0.005851801, 'total_time': 0.029372615}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb348-5e5a-7c23-ae2c-c2cb29772967-0', usage_metadata={'input_tokens': 43, 'output_tokens': 18, 'total_tokens': 61})

### What is this "|" in Python?

In [91]:
from abc import ABC, abstractmethod

class CRunnable(ABC):
    def __init__(self):
        self.next = None

    @abstractmethod
    def process(self, data):
        """
        This method must be implemented by subclasses to define
        data processing behavior.
        """
        pass

    def invoke(self, data):
        processed_data = self.process(data)
        if self.next is not None:
            return self.next.invoke(processed_data)
        return processed_data

    def __or__(self, other):
        return CRunnableSequence(self, other)

class CRunnableSequence(CRunnable):
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second

    def process(self, data):
        return data

    def invoke(self, data):
        first_result = self.first.invoke(data)
        return self.second.invoke(first_result)



In [92]:
class AddTen(CRunnable):
    def process(self, data):
        print("AddTen: ", data)
        return data + 10

class MultiplyByTwo(CRunnable):
    def process(self, data):
        print("Multiply by 2: ", data)
        return data * 2

class ConvertToString(CRunnable):
    def process(self, data):
        print("Convert to string: ", data)
        return f"Result: {data}"

In [93]:
a = AddTen()
b = MultiplyByTwo()
c = ConvertToString()

chain = a | b | c

In [94]:
result = chain.invoke(10)
print(result)

AddTen:  10
Multiply by 2:  20
Convert to string:  40
Result: 40


### Runnables from LangChain

In [95]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [96]:
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

'hello'

In [97]:
def input_to_upper(input: str):
    output = input.upper()
    return output

In [98]:
chain = RunnablePassthrough() | RunnableLambda(input_to_upper) | RunnablePassthrough()
chain.invoke("hello")

'HELLO'

In [99]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})

In [100]:
chain.invoke("hello")

{'x': 'hello', 'y': 'hello'}

In [101]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'},
 'y': {'input': 'hello', 'input2': 'goodbye'}}

In [102]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": lambda z: z["input2"]})

In [103]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'y': 'goodbye'}

### Nested chains - now it gets more complicated!

In [104]:
def find_keys_to_uppercase(input: dict):
    output = input.get("input", "not found").upper()
    return output

In [105]:
chain = RunnableParallel({"x": RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase), "y": lambda z: z["input2"]})

In [106]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': 'HELLO', 'y': 'goodbye'}

In [107]:
chain = RunnableParallel({"x": RunnablePassthrough()})

def assign_func(input):
    return 100

def multiply(input):
    return input * 10

In [108]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}}

In [109]:
chain = RunnableParallel({"x": RunnablePassthrough()}).assign(extra=RunnableLambda(assign_func))

In [110]:
result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'extra': 100}


### Combine multiple chains (incl. coercion)

In [111]:
def extractor(input: dict):
    return input.get("extra", "Key not found")

def cupper(upper: str):
    return str(upper).upper()

new_chain = RunnableLambda(extractor) | RunnableLambda(cupper)

In [112]:
new_chain.invoke({"extra": "test"})

'TEST'

In [113]:
final_chain = chain | new_chain
final_chain.invoke({"input": "hello", "input2": "goodbye"})

'100'

### Real Work example

In [114]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_texts(
    ["Cats love thuna"], embedding=embeddings
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()})
    | prompt
    | ChatGroq(model="llama-3.1-8b-instant", temperature=0, max_retries=2)
    | StrOutputParser()
)

In [115]:
rag_chain.invoke("What do cats like to eat?")

'Cats love thuna.'

In [116]:
RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()}).invoke("What do cats like to eat?")

{'context': 'Cats love thuna', 'question': 'What do cats like to eat?'}

In [117]:
prompt.invoke({"context": "Cats love thuna", "question": "What do cats like to eat?"})

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\nCats love thuna\n\nQuestion: What do cats like to eat?\n', additional_kwargs={}, response_metadata={})])

In [118]:
model.invoke(prompt.invoke({"context": "Cats love thuna", "question": "What do cats like to eat?"}))

AIMessage(content='Cats love thuna.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 60, 'total_tokens': 67, 'completion_time': 0.019370566, 'completion_tokens_details': None, 'prompt_time': 0.006269436, 'prompt_tokens_details': None, 'queue_time': 0.070878072, 'total_time': 0.025640002}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6b5c123dd9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb348-64fa-7361-b328-baab32631d3c-0', usage_metadata={'input_tokens': 60, 'output_tokens': 7, 'total_tokens': 67})